# "Email Campaign Analysis and Optimization"

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load the data
email_table = pd.read_csv('email_table.csv')
email_opened_table = pd.read_csv('email_opened_table.csv')
link_clicked_table = pd.read_csv('link_clicked_table.csv')

In [5]:
print("Email Table:", email_table.shape)
print("Opened Emails:", email_opened_table.shape)
print("Clicked Emails:", link_clicked_table.shape)

Email Table: (100000, 7)
Opened Emails: (10345, 1)
Clicked Emails: (2119, 1)


In [6]:
email_table.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6


In [7]:
email_opened_table.head()

,email_id
0,284534
1,609056
2,220820
3,905936
4,164034


In [8]:
link_clicked_table.head()

,email_id
0,609056
1,870980
2,935124
3,158501
4,177561


In [9]:
# Create target variables
email_table['opened'] = email_table['email_id'].isin(email_opened_table['email_id']).astype(int)
email_table['clicked'] = email_table['email_id'].isin(link_clicked_table['email_id']).astype(int)

In [10]:
# Feature Engineering
email_table['email_text_binary'] = email_table['email_text'].apply(lambda x: 1 if x == 'long text' else 0)
email_table['email_version_binary'] = email_table['email_version'].apply(lambda x: 1 if x == 'personalized' else 0)

In [11]:
# Create time of day feature
email_table['time_of_day'] = pd.cut(
    email_table['hour'], 
    bins=[0, 6, 12, 18, 24], 
    labels=['night', 'morning', 'afternoon', 'evening']
)

In [12]:
# One-hot encode categorical features
categorical_features = ['time_of_day', 'weekday', 'user_country']
email_table_encoded = pd.get_dummies(email_table, columns=categorical_features)

In [13]:
# Calculate overall metrics
total_emails = len(email_table)
opened_emails = sum(email_table['opened'])
clicked_emails = sum(email_table['clicked'])

# Key performance metrics
open_rate = opened_emails / total_emails * 100
click_rate = clicked_emails / total_emails * 100
click_to_open_rate = clicked_emails / opened_emails * 100 if opened_emails > 0 else 0

print(f"Open Rate: {open_rate:.2f}%")
print(f"Click Rate (CTR): {click_rate:.2f}%")
print(f"Click-to-Open Rate (CTOR): {click_to_open_rate:.2f}%")

Open Rate: 10.35%
Click Rate (CTR): 2.12%
Click-to-Open Rate (CTOR): 20.48%


# Exploratory Data Analysis

In [15]:
# Examine effect of email characteristics on performance
performance_by_text = email_table.groupby('email_text').agg({
    'opened': 'mean',
    'clicked': 'mean'
}).reset_index()

performance_by_version = email_table.groupby('email_version').agg({
    'opened': 'mean',
    'clicked': 'mean'
}).reset_index()

performance_by_weekday = email_table.groupby('weekday').agg({
    'opened': 'mean',
    'clicked': 'mean'
}).reset_index()

performance_by_hour = email_table.groupby('hour').agg({
    'opened': 'mean',
    'clicked': 'mean'
}).reset_index()

performance_by_country = email_table.groupby('user_country').agg({
    'opened': 'mean',
    'clicked': 'mean',
    'email_id': 'count'
}).reset_index().rename(columns={'email_id': 'count'})

# Purchase history analysis
email_table['purchase_bucket'] = pd.qcut(email_table['user_past_purchases'], 
                                         q=5, 
                                         labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

performance_by_purchase = email_table.groupby('purchase_bucket').agg({
    'opened': 'mean',
    'clicked': 'mean',
    'email_id': 'count'
}).reset_index().rename(columns={'email_id': 'count'})

# Predictive Modeling
I will build two models: one for predicting opens and another for predicting clicks after an email is opened.

In [17]:
# Prepare features for the model
features = [col for col in email_table_encoded.columns if col not in 
            ['email_id', 'email_text', 'email_version', 'opened', 'clicked', 'purchase_bucket']]

# Model 1: Predict email opening
X = email_table_encoded[features]
y_open = email_table_encoded['opened']

X_train, X_test, y_open_train, y_open_test = train_test_split(X, y_open, test_size=0.2, random_state=42)

# Train the opens model
open_model = GradientBoostingClassifier(random_state=42)
open_model.fit(X_train, y_open_train)

# Evaluate the opens model
open_pred_proba = open_model.predict_proba(X_test)[:, 1]
open_auc = roc_auc_score(y_open_test, open_pred_proba)
print(f"Open Model AUC: {open_auc:.4f}")

# Feature importance for open model
open_feature_importance = pd.DataFrame({
    'feature': features,
    'importance': open_model.feature_importances_
}).sort_values('importance', ascending=False)

# Model 2: Predict clicks for opened emails
opened_emails_df = email_table_encoded[email_table_encoded['opened'] == 1]
X_opened = opened_emails_df[features]
y_click = opened_emails_df['clicked']

X_click_train, X_click_test, y_click_train, y_click_test = train_test_split(
    X_opened, y_click, test_size=0.2, random_state=42
)

# Train the clicks model
click_model = RandomForestClassifier(random_state=42)
click_model.fit(X_click_train, y_click_train)

# Evaluate the clicks model
click_pred_proba = click_model.predict_proba(X_click_test)[:, 1]
click_auc = roc_auc_score(y_click_test, click_pred_proba)
print(f"Click Model AUC: {click_auc:.4f}")

# Feature importance for click model
click_feature_importance = pd.DataFrame({
    'feature': features,
    'importance': click_model.feature_importances_
}).sort_values('importance', ascending=False)

Open Model AUC: 0.6824
Click Model AUC: 0.5620


# Combined Model for CTR Optimization

In [19]:
# Function to predict overall click probability
def predict_click_probability(df, open_model, click_model, features):
    open_probability = open_model.predict_proba(df[features])[:, 1]
    click_probability = click_model.predict_proba(df[features])[:, 1]
    return open_probability * click_probability

# Apply the combined model to test data
X_full_test = email_table_encoded.sample(frac=0.2, random_state=42)
y_full_test = X_full_test['clicked']
X_full_test = X_full_test[features]

# Calculate predicted probabilities
predicted_probabilities = predict_click_probability(X_full_test, open_model, click_model, features)

# Evaluate the combined model
combined_auc = roc_auc_score(y_full_test, predicted_probabilities)
print(f"Combined Model AUC: {combined_auc:.4f}")

Combined Model AUC: 0.8305


# Estimated Improvement in CTR

In [21]:
# Function to simulate optimized campaign
def simulate_optimized_campaign(df, open_model, click_model, features, percentage_selected=20):
    # Calculate click probabilities for all users
    df['click_probability'] = predict_click_probability(df, open_model, click_model, features)
    
    # Random selection (current approach)
    random_sample = df.sample(frac=percentage_selected/100, random_state=42)
    random_ctr = random_sample['clicked'].mean() * 100
    
    # Model-based selection
    optimized_sample = df.nlargest(int(len(df) * percentage_selected/100), 'click_probability')
    optimized_ctr = optimized_sample['clicked'].mean() * 100
    
    improvement = (optimized_ctr - random_ctr) / random_ctr * 100
    
    return {
        'random_ctr': random_ctr,
        'optimized_ctr': optimized_ctr,
        'absolute_improvement': optimized_ctr - random_ctr,
        'relative_improvement': improvement
    }

# Calculate improvement
email_table_encoded['click_probability'] = predict_click_probability(
    email_table_encoded, open_model, click_model, features
)

improvement_results = simulate_optimized_campaign(
    email_table_encoded, open_model, click_model, features, percentage_selected=20
)

print(f"Random Selection CTR: {improvement_results['random_ctr']:.2f}%")
print(f"Optimized Selection CTR: {improvement_results['optimized_ctr']:.2f}%")
print(f"Absolute Improvement: {improvement_results['absolute_improvement']:.2f} percentage points")
print(f"Relative Improvement: {improvement_results['relative_improvement']:.2f}%")

Random Selection CTR: 2.27%
Optimized Selection CTR: 7.74%
Absolute Improvement: 5.47 percentage points
Relative Improvement: 241.50%


# Testing Framework

In [23]:
def design_ab_test(baseline_ctr, expected_lift, alpha=0.05, power=0.8):
    """
    Calculate required sample size for A/B test
    
    Parameters:
    baseline_ctr (float): Current click-through rate
    expected_lift (float): Expected relative improvement
    alpha (float): Significance level
    power (float): Statistical power
    
    Returns:
    dict: Sample sizes and test parameters
    """
    from statsmodels.stats.power import TTestIndPower
    
    # Convert percentages to proportions
    p1 = baseline_ctr / 100
    p2 = p1 * (1 + expected_lift / 100)
    
    # Calculate effect size (Cohen's h)
    effect_size = 2 * (np.arcsin(np.sqrt(p2)) - np.arcsin(np.sqrt(p1)))
    
    # Calculate sample size
    analysis = TTestIndPower()
    sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1)
    
    return {
        'effect_size': effect_size,
        'sample_size_each_group': int(np.ceil(sample_size)),
        'total_sample_size': int(np.ceil(sample_size * 2)),
        'baseline_ctr': baseline_ctr,
        'expected_ctr': baseline_ctr * (1 + expected_lift / 100),
        'alpha': alpha,
        'power': power
    }

# Design an A/B test based on our expected improvement
ab_test_design = design_ab_test(
    baseline_ctr=improvement_results['random_ctr'],
    expected_lift=improvement_results['relative_improvement'] / 2,  # Conservative estimate
    alpha=0.05,
    power=0.8
)

print("A/B Test Design:")
for key, value in ab_test_design.items():
    print(f"{key}: {value}")

A/B Test Design:
effect_size: 0.1488804641920965
sample_size_each_group: 710
total_sample_size: 1419
baseline_ctr: 2.265
expected_ctr: 5.000000000000001
alpha: 0.05
power: 0.8


In [24]:
email_table_encoded.head()

,email_id,email_text,email_version,hour,user_past_purchases,opened,clicked,email_text_binary,email_version_binary,time_of_day_night,...,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,user_country_ES,user_country_FR,user_country_UK,user_country_US,click_probability
0,85120,short_email,personalized,2,5,0,0,0,1,1,...,0,1,0,0,0,0,0,0,1,0.006335
1,966622,long_email,personalized,12,2,1,1,0,1,0,...,0,1,0,0,0,0,0,1,0,0.033080
2,777221,long_email,personalized,11,2,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0.000000
3,493711,short_email,generic,6,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0.002863
4,106887,long_email,generic,14,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.000000


In [25]:
# Extract the weekday columns
weekday_columns = [col for col in email_table_encoded.columns if col.startswith('weekday_')]

# Reconstruct the 'weekday' column from one-hot encoded columns
email_table_encoded['weekday'] = email_table_encoded[weekday_columns].idxmax(axis=1).str.replace('weekday_', '')

In [26]:
country_columns = [col for col in email_table_encoded.columns if col.startswith('user_country_')]
email_table_encoded['user_country'] = email_table_encoded[country_columns].idxmax(axis=1).str.replace('user_country_', '')

In [27]:
# Create purchase buckets: low (0–2), medium (3–5), high (6+)
bins = [-1, 2, 5, float('inf')]
labels = ['Low', 'Medium', 'High']

email_table_encoded['purchase_bucket'] = pd.cut(email_table_encoded['user_past_purchases'], bins=bins, labels=labels)

# Segment Analysis and Insights

In [29]:
# Function to analyze performance across segments
def analyze_segments(df, segment_column):
    segment_analysis = df.groupby(segment_column).agg({
        'opened': ['count', 'mean'],
        'clicked': ['mean'],
        'click_probability': ['mean']
    })
    
    # Calculate potential lift by targeting top segments vs random
    segment_analysis.columns = ['count', 'open_rate', 'ctr', 'predicted_probability']
    segment_analysis['potential_lift'] = segment_analysis['predicted_probability'] / df['click_probability'].mean() - 1
    
    return segment_analysis.sort_values('potential_lift', ascending=False)

# Analyze different segments
weekday_analysis = analyze_segments(email_table_encoded, 'weekday')
hour_analysis = analyze_segments(email_table_encoded, 'hour')
country_analysis = analyze_segments(email_table_encoded, 'user_country')
purchase_analysis = analyze_segments(email_table_encoded, 'purchase_bucket')
text_analysis = analyze_segments(email_table_encoded, 'email_text')
version_analysis = analyze_segments(email_table_encoded, 'email_version')

# Recommendations and Conclusion

In [31]:
# Identify top performing combinations
email_table_encoded['hour_weekday'] = email_table_encoded['hour'].astype(str) + '_' + email_table_encoded['weekday']
time_day_analysis = analyze_segments(email_table_encoded, 'hour_weekday')

# Generate recommendations based on model insights
recommendations = {
    'Best time to send': hour_analysis.iloc[0].name,
    'Best day to send': weekday_analysis.iloc[0].name,
    'Best time-day combination': time_day_analysis.iloc[0].name,
    'Email format': text_analysis.iloc[0].name,
    'Personalization': version_analysis.iloc[0].name,
    'Priority countries': country_analysis.iloc[:3].index.tolist(),
    'Purchase history targeting': purchase_analysis.iloc[0].name
}